In [18]:
from ultralytics import YOLO

In [19]:
model = YOLO("yolo11n.pt")

In [20]:
result = model("car.jpg")[0]


image 1/1 D:\00.MOTHER_BOXES\code\modification\car.jpg: 480x640 1 car, 114.7ms
Speed: 6.1ms preprocess, 114.7ms inference, 19.3ms postprocess per image at shape (1, 3, 480, 640)


In [21]:
result[0].show()

In [22]:
from ultralytics.models.yolo.detect.predict import DetectionPredictor
from ultralytics import YOLO

In [23]:
# Load your model
model = YOLO("yolo11n.pt")

In [24]:
# Create predictor directly
predictor = DetectionPredictor(overrides={
    "conf": 0.25,
    "imgsz": 640,
    "device": "cuda"  # or "cuda"
})

In [25]:
# Setup the model (CRITICAL STEP - this is what the Model class does internally)
predictor.setup_model(model=model.model, verbose=True)

Ultralytics 8.3.184  Python-3.13.5 torch-2.9.0+cu130 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
YOLO11n summary (fused): 100 layers, 2,616,248 parameters, 0 gradients


In [26]:
# Now you can use it directly
results = predictor("car.jpg")


image 1/1 D:\00.MOTHER_BOXES\code\modification\car.jpg: 640x640 1 car, 61.7ms
Speed: 6.7ms preprocess, 61.7ms inference, 2.8ms postprocess per image at shape (1, 3, 640, 640)
Results saved to D:\00.MOTHER_BOXES\code\modification\runs\detect\train13


In [27]:
results[0].show()

preprocessings

In [28]:
from ultralytics import YOLO
from ultralytics.engine.predictor import BasePredictor

In [29]:
# 2. Build predictor (this contains YOLO preprocess)
predictor = BasePredictor(overrides={"model": "yolo11n.pt"})

In [30]:
# 3. Manually set up model inside predictor
predictor.setup_model("yolo11n.pt")

Ultralytics 8.3.184  Python-3.13.5 torch-2.9.0+cu130 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
YOLO11n summary (fused): 100 layers, 2,616,248 parameters, 0 gradients


In [31]:
# 4. Load your JPG image using YOLO’s own loader
dataset = predictor.setup_source("car.jpg")

In [32]:
# 5. Get raw image from predictor as YOLO sees it
paths, im0s, _ = next(iter(predictor.dataset))

In [33]:
# 6. NOW apply YOLO’s EXACT preprocess
img_tensor = predictor.preprocess(im0s)

In [34]:
print(img_tensor.shape)

torch.Size([1, 3, 640, 640])


In [35]:
preds = predictor.model(img_tensor)

In [36]:
len(preds), type(preds[0]), type(preds[1])

(2, torch.Tensor, list)

In [37]:
len(preds[1]), preds[1][0].shape, preds[1][1].shape, preds[1][2].shape

(3,
 torch.Size([1, 144, 80, 80]),
 torch.Size([1, 144, 40, 40]),
 torch.Size([1, 144, 20, 20]))

In [38]:
preds[0].shape

torch.Size([1, 84, 8400])

In [39]:
from ultralytics.utils.ops import non_max_suppression
from ultralytics.engine.results import Results

# 1) apply YOLO's NMS (same used inside AutoBackend)
nms_output = non_max_suppression(
    preds,
    conf_thres=predictor.args.conf,
    iou_thres=predictor.args.iou,
    max_det=predictor.args.max_det,
    classes=None,
    agnostic=predictor.args.agnostic_nms
)

In [40]:
nms_output

[tensor([[231.8356, 260.1294, 584.2322, 420.9265,   0.8156,   2.0000]], device='cuda:0')]

In [41]:
print(predictor.args.conf, predictor.args.iou, predictor.args.max_det, predictor.args.agnostic_nms, sep= "\n")

0.25
0.7
300
False


In [42]:
# 2) Construct YOLO Results objects for each image
results = [
    Results(
        orig_img=im0s[i],
        path=paths[i],
        names=predictor.model.names,
        boxes=nms_output[i]
    )
    for i in range(len(nms_output))
]

In [43]:
im0s[0].shape

(1530, 2048, 3)

In [44]:
import cv2
import torch

# Your raw NMS output
detections = [torch.tensor([[231.8356, 260.1294, 584.2322, 420.9265, 0.8156, 2.0000]], device='cuda:0')]

# Load your image
img = cv2.imread("car.jpg")
img = cv2.resize(img, (640, 640))
# Loop through detections
for det in detections[0].cpu().numpy():   # move to CPU and numpy
    x1, y1, x2, y2, conf, cls = det

    # Convert to int
    x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])

    # Draw bounding box
    cv2.rectangle(
        img,
        (x1, y1),
        (x2, y2),
        (0, 255, 0),   # green box
        2              # box thickness
    )

    # Draw label text
    label = f"{int(cls)} {conf:.2f}"
    cv2.putText(
        img,
        label,
        (x1, y1 - 10),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.7,            # font size
        (0, 255, 0),    # text color
        2               # thickness
    )

# Save or show result
# cv2.imwrite("result.jpg", img)
cv2.imshow("result", img); cv2.waitKey(0)

-1